In [ ]:
#Downloading gspread library to access files from drive
!pip install --upgrade gspread
!pip install gspread_dataframe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: gspread
    Found existing installation: gspread 6.0.2
    Uninstalling gspread-6.0.2:
      Successfully uninstalled gspread-6.0.2


In [ ]:
#connecting colab with drive
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
import pandas as pd

# Import the first sheet of a Google Sheet
sheet_url = "https://docs.google.com/spreadsheets/d/1iwGiXmyyTJgl6C8oZknQ5ACnj5zo4GAmrgv3i46OD3o/edit?usp=sharing"
sheet = gc.open_by_url(sheet_url).sheet1  # Access the first sheet
counties = pd.DataFrame(sheet.get_all_records())

# Display the first rows of the DataFrame
counties.head()

,year,state_po,state,county_name,DEMOCRAT,GREEN,LIBERTARIAN,OTHER,REPUBLICAN
0,2020,AK,ALASKA,DISTRICT6,3272,83,266,69,5770
1,2020,AK,ALASKA,DISTRICT32,3506,105,238,46,4440
2,2020,AK,ALASKA,DISTRICT35,5011,95,170,57,4769
3,2020,AK,ALASKA,DISTRICT36,3796,79,245,58,5114
4,2020,AK,ALASKA,DISTRICT37,2560,56,86,82,2358


In [ ]:
counties = counties.drop(columns=['year',	'state'])
counties

,state_po,county_name,DEMOCRAT,GREEN,LIBERTARIAN,OTHER,REPUBLICAN
0,AK,DISTRICT6,3272,83,266,69,5770
1,AK,DISTRICT32,3506,105,238,46,4440
2,AK,DISTRICT35,5011,95,170,57,4769
3,AK,DISTRICT36,3796,79,245,58,5114
4,AK,DISTRICT37,2560,56,86,82,2358
...,...,...,...,...,...,...,...
3123,WY,SWEETWATER,3823,0,350,296,12229
3124,WY,TETON,9848,0,255,343,4341
3125,WY,UINTA,1591,0,172,200,7496
3126,WY,WASHAKIE,651,0,65,71,3245


In [ ]:
#importing GDP sheet
sheet_url_GDP = "https://docs.google.com/spreadsheets/d/1bSihrUMWV2CBkK_XHiZG2wdxe2CZM5n9DYy5GOBitc0/edit?usp=sharing"
GDP_sheet = gc.open_by_url(sheet_url_GDP).sheet1
GDP = pd.DataFrame(GDP_sheet.get_all_records())
# Display the first rows of the DataFrame
GDP.head()

,Fips,state postal code,county,Real GDP
0,1001,,"Autauga, AL",1746979
1,1003,,"Baldwin, AL",8109210
2,1005,,"Barbour, AL",732151
3,1007,,"Bibb, AL",460844
4,1009,,"Blount, AL",906582


In [ ]:
#importing unemployment sheet
sheet_url_Unemployment = "https://docs.google.com/spreadsheets/d/1SC7S02xCKgMvE0FurOzBKoOk0LyZNKAAGMtx42ESUBs/edit?usp=sharing"
Unemployment_sheet = gc.open_by_url(sheet_url_Unemployment).sheet1
Unemp = pd.DataFrame(Unemployment_sheet.get_all_records())
# Display the first rows of the DataFrame
Unemp.head()


,FIPS_Code,state_po,county,unemp rate
0,2000,AK,Alaska,8.3
1,2013,AK,DISTRICT6,10.8201
2,2016,AK,DISTRICT32,6.7
3,2020,AK,DISTRICT35,8.849941
4,2050,AK,DISTRICT36,9.996972


In [ ]:
#importing poverty sheet
sheet_url_poverty = "https://docs.google.com/spreadsheets/d/1RaS19fn3KSVKp8fz3gZ8WyCMDXUEnr-wVd8NAXZw8t0/edit?usp=sharing"
poverty_sheet = gc.open_by_url(sheet_url_poverty).sheet1
poverty = pd.DataFrame(poverty_sheet.get_all_records())
# Display the first rows of the DataFrame
poverty = poverty[poverty['County FIPS Code'] != 0]
poverty.head()



,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Percent, All Ages"
2,1,1,AL,Autauga County,11.2
3,1,3,AL,Baldwin County,8.9
4,1,5,AL,Barbour County,25.5
5,1,7,AL,Bibb County,17.8
6,1,9,AL,Blount County,13.1


In [ ]:
# Normalize county name function
def normalize_county_name(name):
    # Convert to lowercase
    name = name.lower()
    # Remove specific keywords
    name = name.replace("county", "").replace("borough", "").replace("census area", "")
    # Remove non-alphanumeric characters except spaces
    name = ''.join(e for e in name if e.isalnum() or e.isspace()).strip()
    return name

# Extract state code and normalize county names
GDP['state postal code'] = GDP['county'].str.extract(r',\s*([A-Z]{2})\*?')  # Extract state code (handles optional *)
GDP['county'] = GDP['county'].str.replace(r',\s*[A-Z]{2}\*?', '', regex=True)  # Remove state code from County
GDP['county'] = GDP['county'].apply(normalize_county_name)  # Normalize the county name
GDP = GDP.drop(columns=['Fips'])

GDP

,state postal code,county,Real GDP
0,AL,autauga,1746979
1,AL,baldwin,8109210
2,AL,barbour,732151
3,AL,bibb,460844
4,AL,blount,906582
...,...,...,...
3092,WY,sweetwater,3404245
3093,WY,teton,2688047
3094,WY,uinta,781034
3095,WY,washakie,342040


In [ ]:
counties['county'] = counties['county'].apply(normalize_county_name)  # Normalize the county name
counties

,state_po,county,DEMOCRAT,GREEN,LIBERTARIAN,OTHER,REPUBLICAN
0,AK,district6,3272,83,266,69,5770
1,AK,district32,3506,105,238,46,4440
2,AK,district35,5011,95,170,57,4769
3,AK,district36,3796,79,245,58,5114
4,AK,district37,2560,56,86,82,2358
...,...,...,...,...,...,...,...
3123,WY,sweetwater,3823,0,350,296,12229
3124,WY,teton,9848,0,255,343,4341
3125,WY,uinta,1591,0,172,200,7496
3126,WY,washakie,651,0,65,71,3245


In [ ]:
#Removing states to keep only counties
unemploy = Unemp[Unemp['county'].str.endswith(tuple([f', {state}' for state in ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']]))]

# Reset the index after deleting rows
unemploy = unemploy.reset_index(drop=True)

unemploy['county'] = unemploy['county'].str.replace(r',\s*[A-Z]{2}\*?', '', regex=True)  # Remove state code from County
unemploy['county'] = unemploy['county'].apply(normalize_county_name)  # Normalize the county name
unemploy = unemploy.drop(columns=['FIPS_Code'])

unemploy

,state_po,county,unemp rate
0,AL,autauga,5.3
1,AL,baldwin,6.1
2,AL,barbour,7.7
3,AL,bibb,7.3
4,AL,blount,4.5
...,...,...,...
3106,WY,sweetwater,7.5
3107,WY,teton,6.0
3108,WY,uinta,6.4
3109,WY,washakie,5.3


In [ ]:
poverty['Name'] = poverty['Name'].apply(normalize_county_name)  # Normalize the county name
# Drop the 'State FIPS Code' column as it is no longer needed
poverty = poverty.drop(columns=['State FIPS Code','County FIPS Code'])
poverty = poverty.reset_index(drop=True)

poverty

,Postal Code,Name,"Poverty Percent, All Ages"
0,AL,autauga,11.2
1,AL,baldwin,8.9
2,AL,barbour,25.5
3,AL,bibb,17.8
4,AL,blount,13.1
...,...,...,...
3121,WY,sweetwater,7.6
3122,WY,teton,5.2
3123,WY,uinta,8.5
3124,WY,washakie,9.7


In [ ]:
#Standardize the column names

poverty.rename(columns={"Name": "county"}, inplace=True)
counties.rename(columns={"county_name": "county"}, inplace=True)

poverty.rename(columns={"Postal Code": "state_po"}, inplace=True)
GDP.rename(columns={"state postal code": "state_po"}, inplace=True)

counties_sorted = counties.sort_values(by='county').reset_index(drop=True)
unemploy_sorted = unemploy.sort_values(by='county').reset_index(drop=True)
poverty_sorted = poverty.sort_values(by='county').reset_index(drop=True)
GDP_sorted = GDP.sort_values(by='county').reset_index(drop=True)

# Check the result
print(counties_sorted.head())
print(unemploy_sorted.head())
print(poverty_sorted.head())
print(GDP_sorted.head())

  state_po     county DEMOCRAT GREEN LIBERTARIAN OTHER REPUBLICAN
0       SC  abbeville     4101    23          88     6       8215
1       LA     acadia     5443     0         241   145      22596
2       VA   accomack     7578     0         188    24       9172
3       ID        ada   120539     0        5310  2841     130699
4       OK      adair     1387     0          62    74       5585
  state_po         county unemp rate
0       SC      abbeville        6.5
1       LA  acadia parish        7.7
2       VA       accomack        6.7
3       ID            ada        5.5
4       OK          adair        6.0
  state_po         county Poverty Percent, All Ages
0       SC      abbeville                      15.3
1       LA  acadia parish                      20.7
2       VA       accomack                      17.6
3       ID            ada                       7.7
4       OK          adair                      22.3
  state_po     county  Real GDP
0       SC  abbeville    572455
1     

In [ ]:
# Left join to keep all counties from the first dataset
merged_df = pd.merge(counties_sorted, unemploy_sorted, left_on='county', right_on='county', how='left')
merged_df = pd.merge(merged_df, poverty_sorted, left_on='county', right_on='county', how='left')
merged_df = pd.merge(merged_df, GDP_sorted, left_on='county', right_on='county', how='left')

# Replace missing values with a placeholder or zero
merged_df.fillna(0, inplace=True)

MergeError: Passing 'suffixes' which cause duplicate columns {'state_po_x'} is not allowed.

In [ ]:
merged_df = pd.merge(counties_sorted, unemploy_sorted, left_on='county', right_on='county', how='inner')
merged_df = pd.merge(merged_df, poverty_sorted, left_on='county', right_on='county', how='inner')
merged_df = pd.merge(merged_df, GDP_sorted, left_on='county', right_on='county', how='inner')

merged_df

MergeError: Passing 'suffixes' which cause duplicate columns {'state_po_x'} is not allowed.

In [ ]:
# First merge (retain only one set of 'county' and 'state_po' columns)
merged_df = pd.merge(
    counties_sorted,
    unemploy_sorted,
    on=['county', 'state_po'],
    how='inner',
    suffixes=('', '_unemploy')
)

# Second merge
merged_df = pd.merge(
    merged_df,
    poverty_sorted,
    on=['county', 'state_po'],
    how='inner',
    suffixes=('', '_poverty')
)

# Third merge
merged_df = pd.merge(
    merged_df,
    GDP_sorted,
    on=['county', 'state_po'],
    how='inner',
    suffixes=('', '_GDP')
)

# Dropping the duplicate columns created during the merge
for col in merged_df.columns:
    if col.endswith(('_unemploy', '_poverty', '_GDP')):
        original_col = col.rsplit('_', 1)[0]
        if original_col in merged_df.columns:
            merged_df.drop(col, axis=1, inplace=True)

# Fill missing values with zeros or placeholders if necessary
merged_df.fillna(0, inplace=True)

# View the merged DataFrame
print(merged_df.head())
merged_df

  state_po     county DEMOCRAT GREEN LIBERTARIAN OTHER REPUBLICAN unemp rate  \
0       SC  abbeville     4101    23          88     6       8215        6.5   
1       VA   accomack     7578     0         188    24       9172        6.7   
2       ID        ada   120539     0        5310  2841     130699        5.5   
3       OK      adair     1387     0          62    74       5585        6.0   
4       MO      adair     3710    43         157    13       6413        5.1   

  Poverty Percent, All Ages  Real GDP  
0                      15.3    572455  
1                      17.6   1785338  
2                       7.7  29310868  
3                      22.3    395223  
4                      17.9    785869  


,state_po,county,DEMOCRAT,GREEN,LIBERTARIAN,OTHER,REPUBLICAN,unemp rate,"Poverty Percent, All Ages",Real GDP
0,SC,abbeville,4101,23,88,6,8215,6.5,15.3,572455
1,VA,accomack,7578,0,188,24,9172,6.7,17.6,1785338
2,ID,ada,120539,0,5310,2841,130699,5.5,7.7,29310868
3,OK,adair,1387,0,62,74,5585,6.0,22.3,395223
4,MO,adair,3710,43,157,13,6413,5.1,17.9,785869
...,...,...,...,...,...,...,...,...,...,...
2963,CO,yuma,785,2,52,35,4107,2.6,11.5,400574
2964,AZ,yuma,32210,0,1137,0,36534,17.1,15.2,7571543
2965,TX,zapata,1826,4,11,0,2033,12.3,24.6,551863
2966,TX,zavala,2864,12,13,0,1490,14.3,27.2,569285


In [ ]:
from google.colab import files

# Save the merged DataFrame as an Excel file
merged_df.to_excel("merged_data.xlsx", index=False)

# Download the Excel file
files.download("merged_data.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install openpyxl

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Specify the path where you want to save the file in your Google Drive
output_path = 'https://drive.google.com/drive/u/0/folders/1M3nSo-xvrGVwcIRbxQ2tPlSPnpLoq0pg'

# Save the merged DataFrame as an Excel file
try:
    merged_df.to_excel(output_path, index=False, engine='openpyxl')
    print(f"File successfully saved to: {output_path}")
except Exception as e:
    print(f"An error occurred: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
File successfully saved to: https://drive.google.com/drive/u/0/folders/1M3nSo-xvrGVwcIRbxQ2tPlSPnpLoq0pg
